In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] =10,6

import textwrap

In [ ]:
import plotly.express as px
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn import preprocessing
import optuna
from optuna.samplers import TPESampler

In [ ]:
train_df = pd.read_csv('../input/av-healthcare-analytics-ii/healthcare/train_data.csv')
test_df =  pd.read_csv('../input/av-healthcare-analytics-ii/healthcare/test_data.csv')
sample_df =  pd.read_csv('../input/av-healthcare-analytics-ii/healthcare/sample_sub.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
test_df.head()

In [ ]:
print("Shape of Training Data ... "+str(train_df.shape))
print("Shape of Testing Data ... "+str(test_df.shape))

In [ ]:
# Genersting the pie chart
def pie_chart(df,col,path):
  label = df[col].value_counts().index.tolist()
  fig = plt.figure(figsize=(10,6))
  ax = (df[col].value_counts()*100.0 /len(df))\
  .plot.pie(startangle=90,autopct='%.1f%%', labels =label, fontsize=12)                                                                           
  ax.set_title('% '+str(col))
  # plt.savefig(path+str(col1)+'.png')
  plt.show()
    
# Relation between the categorical variable    
def rel_cat(df,x_axis,y_axis,path,stacked=None):
    temp =pd.crosstab(df[x_axis],df[y_axis])
    temp.plot(kind='bar',stacked=stacked,grid=False)
    plt.xlabel(str(x_axis),weight='bold',fontsize=12)
    plt.ylabel(str(y_axis),weight='bold',fontsize=12)
    plt.title(str(x_axis)+'_'+'and'+'_'+str(y_axis),weight='bold',fontsize=14)
    plt.xticks(rotation=0,fontsize=12)
    plt.yticks(fontsize=12)
    labels = df[x_axis].value_counts().index.tolist()
    labels.sort()
    labels=[textwrap.fill(text,10) for text in labels]
    pos = np.arange(len(labels)) 
    plt.xticks(pos, labels)
#     plt.legend()
#     plt.savefig(path+str(x_axis)+'_'+'and'+'_'+str(y_axis)+'.jpg')    
    plt.show()

In [ ]:
train_df.isnull().sum()


In [ ]:
pie_chart(train_df,'Hospital_type_code',8)


In [ ]:
# sns.set(font_scale = 1.5)
# plt.figure(figsize=(15,8))
ax = sns.countplot(x='Stay',data=train_df,palette='winter_r')
plt.xlabel("Category of Stay",weight='bold',fontsize=18)
plt.ylabel("Count Category of Stay",weight='bold',fontsize=18)
plt.title("Count of Stay by each category",weight='bold',fontsize=20)

#adding the text labels
rects = ax.patches
labels = train_df['Stay'].value_counts().tolist()
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom', fontsize=14)

c_labels = train_df['Stay'].value_counts().index.tolist()
c_name=[textwrap.fill(text,10) for text in c_labels]
pos = np.arange(len(c_name)) 
plt.xticks(pos, c_name)

plt.show


In [ ]:
# Check the cardinality of each columns
cardinality_df = pd.DataFrame()
l = []
for i in range(len(train_df.columns)):
#     print(i)
    distinct_count = train_df[train_df.columns[i]].value_counts().count()
    l.append(distinct_count)
col_name = train_df.columns.tolist()
cardinality_df["Columns_Name"]= col_name
cardinality_df['Cardinality_Count'] = l
cardinality_df

In [ ]:
pie_chart(train_df,'Severity of Illness',8)

In [ ]:
# sns.set(font_scale = 1)
# plt.figure(figsize=(10,6))
t1 = train_df.groupby(['Type of Admission','Severity of Illness'])[['Stay']].count().add_prefix('count_of_').reset_index()
t2 = t1.pivot('Type of Admission','Severity of Illness','count_of_Stay')
t2.plot(kind='bar',stacked=True)
plt.legend(loc='best')
plt.xticks(rotation=0,fontsize=12)
plt.xlabel("Category of Stay",weight='bold',fontsize=18)
plt.ylabel("Count Category of Stay",weight='bold',fontsize=18)
plt.title("Type of Admission, Severity of illnes and Stay",weight='bold',fontsize=20)
plt.show()

In [ ]:
rel_cat(train_df,'Type of Admission','Severity of Illness',8,stacked=None)


In [ ]:
def bar_chart(df,parameter, figsize=(10,6)):
    target_counts = df[parameter].value_counts()
    target_perc = target_counts.div(target_counts.sum(), axis=0)
    plt.figure(figsize=figsize)
    ax = sns.barplot(x=target_counts.index.values, y=target_counts.values, order=target_counts.index,palette='winter')
#     plt.xticks(rotation=90)
    plt.xlabel(f'{parameter}', weight ='bold',fontsize=16)
    plt.ylabel('# of occurances', weight ='bold',fontsize=16)
    plt.title("Count of "+f'{parameter}', weight ='bold',fontsize=20)

    rects = ax.patches
    labels = np.round(target_perc.values*100, 2)
    for rect, label in zip(rects, labels):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2, height + 5, f'{label}%', ha='center', va='bottom')
    
    try:
        labels =target_counts.index.tolist()

    #     labels.sort()
        labels=[textwrap.fill(text,12) for text in labels]
        pos = np.arange(len(labels)) 
        plt.xticks(pos, labels,fontsize=12)
    except:
        pass

In [ ]:
bar_chart(train_df,'Stay')

In [ ]:
bar_chart(train_df,'Department')

In [ ]:
bar_chart(train_df ,'Available Extra Rooms in Hospital', figsize=(20, 4))

In [ ]:
# get_info('Ward_Type')
bar_chart(train_df,'Ward_Type')

In [ ]:
bar_chart(train_df ,'Age')

In [ ]:
bar_chart(train_df,'Type of Admission')

In [ ]:
bar_chart(train_df,'Severity of Illness')

In [ ]:
bar_chart(train_df,'Bed Grade')

In [ ]:
bar_chart(train_df,'Hospital_region_code')

In [ ]:
sns.distplot(train_df['Admission_Deposit'], kde=False, bins=50,color='b')
plt.xlabel("Admission_Deposit",weight='bold',fontsize=18)
plt.ylabel(" Frequency Admission_Deposit",weight='bold',fontsize=18)
plt.title("Admission_Deposit",weight='bold',fontsize=20)
plt.show()

In [ ]:
order = ['0-10', '11-20', '21-30', '31-40', 
         '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '100+']

In [ ]:
sns.catplot(x="Stay", hue="Bed Grade", kind="count",
            palette="pastel", edgecolor=".6",
            data=train_df, height=5, aspect=2, order=order)
plt.xticks(rotation=0)
plt.xlabel("Stay",weight='bold',fontsize=18)
plt.ylabel(" Count",weight='bold',fontsize=18)
plt.title("Relation between Bed Grade and Stay",weight='bold',fontsize=20)
plt.show()

In [ ]:
sns.catplot(x="Stay", hue="Severity of Illness", kind="count",
            palette="pastel", edgecolor=".9",
            data=train_df, height=5, aspect=2, order=order)
plt.xticks(rotation=0)
plt.xlabel("Stay",weight='bold',fontsize=18)
plt.ylabel(" Count ",weight='bold',fontsize=18)
plt.title("Relation between Stay and Severity of Illness",weight='bold',fontsize=20)
plt.show()


In [ ]:
sns.catplot(x='Stay', hue="Type of Admission", kind="count",
            palette="winter_r", edgecolor=".6",
            data=train_df, height=5, aspect=2, order=order)
plt.xticks(rotation=0)
plt.xlabel("Stay",weight='bold',fontsize=18)
plt.ylabel(" Count ",weight='bold',fontsize=18)
plt.title("Relation between Stay and Type of Admission",weight='bold',fontsize=20)
plt.show()


In [ ]:
sns.catplot(x='Stay', hue="Hospital_region_code", kind="count",
            palette="Pastel2", edgecolor=".6",
            data=train_df, height=5, aspect=2, order=order)
plt.xticks(rotation=0)
plt.xlabel("Stay",weight='bold',fontsize=18)
plt.ylabel(" Count ",weight='bold',fontsize=18)
plt.title("Relation between Stay and Hospital_region_code",weight='bold',fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(14,8))
sns.boxplot(y='Visitors with Patient',x='Stay', hue="Severity of Illness", order=order, data=train_df)
plt.xticks(fontsize=12)
plt.xlabel("Stay",weight='bold',fontsize=18)
plt.ylabel(" Visitors with Patient ",weight='bold',fontsize=18)
plt.title("Boxplot between Stay,Visitors with Patient by Severity of Illness",weight='bold',fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(14,8))
sns.boxplot(x='Stay',y='Admission_Deposit', order=order, data=train_df,palette='winter')
plt.xticks(fontsize=12)
plt.xlabel("Stay",weight='bold',fontsize=18)
plt.ylabel(" Admission_Deposit ",weight='bold',fontsize=18)
plt.title("Boxplot between Stay and Admission_Deposit",weight='bold',fontsize=20)
plt.show()

In [ ]:
sns.catplot(x='Stay', hue="Ward_Facility_Code", kind="count",
            palette="pastel", edgecolor=".6",
            data=train_df, height=5, aspect=2, order=order)
plt.xticks(rotation=0)
plt.xticks(fontsize=12)
plt.xlabel("Stay",weight='bold',fontsize=18)
plt.ylabel("Count",weight='bold',fontsize=18)
plt.title("Boxplot between Stay and Ward_Facility_Code",weight='bold',fontsize=20)
plt.show()

In [ ]:
sns.heatmap(train_df.corr())
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(14, 8))
ax = sns.countplot(x="Age", hue="Type of Admission", data=train_df,palette='Pastel2')
plt.xticks(rotation=0)
plt.xticks(fontsize=12)
plt.xlabel("Age",weight='bold',fontsize=18)
plt.ylabel("Count",weight='bold',fontsize=18)
plt.title("Relation between Age and Type of  Admission",weight='bold',fontsize=20)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(14,8))
ax = sns.countplot(x="Age", hue="Severity of Illness", data=train_df,palette='coolwarm')
plt.xticks(rotation=0)
plt.xticks(fontsize=12)
plt.xlabel("Age",weight='bold',fontsize=18)
plt.ylabel("Count",weight='bold',fontsize=18)
plt.title("Relation between Age and Severity of Illness",weight='bold',fontsize=20)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(14,8))
ax = sns.countplot(x="Age", hue="Department", data=train_df,palette='brg')
plt.xticks(rotation=0)
plt.xticks(fontsize=12)
plt.xlabel("Age",weight='bold',fontsize=18)
plt.ylabel("Count",weight='bold',fontsize=18)
plt.title("Relation between Age and Department",weight='bold',fontsize=20)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(14,8))
ax = sns.countplot(x="Age", hue="Stay", data=train_df,palette='pastel')
plt.xticks(rotation=0)
plt.xticks(fontsize=12)
plt.xlabel("Age",weight='bold',fontsize=18)
plt.ylabel("Count",weight='bold',fontsize=18)
plt.title("Relation between Age and Stay",weight='bold',fontsize=20)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(14,8))
ax = sns.countplot(x="Type of Admission", hue="Stay", data=train_df,palette='winter')
plt.xticks(rotation=0)
plt.xticks(fontsize=12)
plt.xlabel("Type of Admission",weight='bold',fontsize=18)
plt.ylabel("Count",weight='bold',fontsize=18)
plt.title("Relation Type of Admission and Stay",weight='bold',fontsize=20)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(14,8))
ax = sns.countplot(x="Severity of Illness", hue="Stay", data=train_df)
plt.xticks(rotation=0)
plt.xticks(fontsize=12)
plt.xlabel("Severity of Illness",weight='bold',fontsize=18)
plt.ylabel("Count",weight='bold',fontsize=18)
plt.title("Relation Type of Severity of Illness and Stay",weight='bold',fontsize=20)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(14,8))
ax = sns.countplot(x="Department", hue="Stay", data=train_df,palette='pastel')
plt.xticks(rotation=0)
plt.xticks(fontsize=12)
plt.xlabel("Department",weight='bold',fontsize=18)
plt.ylabel("Count",weight='bold',fontsize=18)
plt.title("Relation Department and Stay",weight='bold',fontsize=20)
plt.show()

In [ ]:
def correlation_matrix(df):
    from matplotlib import pyplot as plt
    from matplotlib import cm as cm

    fig = plt.figure(figsize=(16,12))
    ax1 = fig.add_subplot(111)
    cmap = cm.get_cmap('jet', 30)
    cax = ax1.imshow(df.corr(), interpolation="nearest", cmap=cmap)
    ax1.grid(True)
    plt.title('Health Care Analytics dataset features correlation\n',weight='bold',fontsize=20)
    labels=df.columns
#     ax1.set_xticklabels(labels,fontsize=9)
    ax1.set_yticklabels(labels,fontsize=12)
    labels =df.columns.tolist()

    #     labels.sort()
    labels=[textwrap.fill(text,10) for text in labels]
    pos = np.arange(len(labels)) 
    ax1.set_xticklabels(labels,fontsize=12)
    # Add colorbar, make sure to specify tick locations to match desired ticklabels
    fig.colorbar(cax, ticks=[0.1*i for i in range(-11,11)])
    plt.show()


In [ ]:
correlation_matrix(train_df)

In [ ]:
#Plotting data 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px

plt.figure(figsize=(20,14))
sns.heatmap(train_df.corr(),annot=True,linecolor='white',linewidths=2,cmap = 'plasma')

In [ ]:
train = train_df.drop(['case_id'], axis=1)
test = test_df.drop(['case_id'], axis=1)
train['dataset'] = 'train'
test['dataset'] = 'test'
df = pd.concat([train, test])

In [ ]:
df.loc[df['Stay'] == '0-10', 'Stay'] = 0
df.loc[df['Stay'] == '11-20', 'Stay'] = 1
df.loc[df['Stay'] == '21-30', 'Stay'] = 2
df.loc[df['Stay'] == '31-40', 'Stay'] = 3
df.loc[df['Stay'] == '41-50', 'Stay'] = 4
df.loc[df['Stay'] == '51-60', 'Stay'] = 5
df.loc[df['Stay'] == '61-70', 'Stay'] = 6
df.loc[df['Stay'] == '71-80', 'Stay'] = 7
df.loc[df['Stay'] == '81-90', 'Stay'] = 8
df.loc[df['Stay'] == '91-100', 'Stay'] = 9
df.loc[df['Stay'] == 'More than 100 Days', 'Stay'] = 10

In [ ]:
train = df[df['dataset']=='train']
test = df[df['dataset']=='test']

target = train['Stay']

features = ['Available Extra Rooms in Hospital', 'Bed Grade', 'Visitors with Patient', 'Admission_Deposit']

train = train[features]
train = train.fillna(0)
test = test[features]

In [ ]:
X, X_val, y, y_val = train_test_split(train, target, random_state=0, test_size=0.2, shuffle=True)
y=y.astype('int')
y_val=y_val.astype('int')

In [ ]:
model = LogisticRegression(random_state=666)
model.fit(X, y)
preds = model.predict(X_val)
print('Baseline accuracy: ', accuracy_score(y_val, preds)*100, '%')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plot_confusion_matrix(model, X_val, y_val, ax=ax)

In [ ]:
need_to_encode = ['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness']
for column in need_to_encode:
    le = preprocessing.LabelEncoder()
    le.fit(df[column])
    df[column] = le.transform(df[column])

In [ ]:
df.loc[df['Age'] == '0-10', 'Age'] = 0
df.loc[df['Age'] == '11-20', 'Age'] = 1
df.loc[df['Age'] == '21-30', 'Age'] = 2
df.loc[df['Age'] == '31-40', 'Age'] = 3
df.loc[df['Age'] == '41-50', 'Age'] = 4
df.loc[df['Age'] == '51-60', 'Age'] = 5
df.loc[df['Age'] == '61-70', 'Age'] = 6
df.loc[df['Age'] == '71-80', 'Age'] = 7
df.loc[df['Age'] == '81-90', 'Age'] = 8
df.loc[df['Age'] == '91-100', 'Age'] = 9

In [ ]:
categorical = ['Hospital_code', 'Hospital_type_code', 'City_Code_Hospital', 'Hospital_region_code', 'Department', 'Ward_Type', 'Ward_Facility_Code', 
              'City_Code_Patient', 'Type of Admission', 'Severity of Illness']

In [ ]:
train = df[df['dataset']=='train']
test = df[df['dataset']=='test']

target = train['Stay']
train = train.fillna(0)
test = test.fillna(0)
train = train.drop(['patientid', 'dataset', 'Stay'], axis=1)
test = test.drop(['patientid', 'dataset'], axis=1)
train

In [ ]:
X, X_val, y, y_val = train_test_split(train, target, random_state=0, test_size=0.2, shuffle=True)
y=y.astype('int')
y_val=y_val.astype('int')

In [ ]:
model = LGBMClassifier(random_state=666)
model.fit(X, y, categorical_feature=categorical)
preds = model.predict(X_val)
print('LGBM accuracy: ', accuracy_score(y_val, preds)*100, '%')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plot_confusion_matrix(model, X_val, y_val, ax=ax)

In [ ]:
#We can see that we improved our score without any serious preprocessing of data and hyperparameters tunning. Let's do it next.
#Optuna optimization
sampler = TPESampler(seed=0)
def create_model(trial):
    max_depth = trial.suggest_int("max_depth", 2, 30)
    n_estimators = trial.suggest_int("n_estimators", 1, 500)
    learning_rate = trial.suggest_uniform('learning_rate', 0.0000001, 1)
    num_leaves = trial.suggest_int("num_leaves", 2, 5000)
    min_child_samples = trial.suggest_int('min_child_samples', 3, 200)
    model = LGBMClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth, num_leaves=num_leaves, min_child_samples=min_child_samples,
                           random_state=0)
    return model

def objective(trial):
    model = create_model(trial)
    model.fit(X, y)
    preds = model.predict(X_val)
    return accuracy_score(y_val, preds)

def optimize():
    study = optuna.create_study(direction="maximize", sampler=sampler)
    study.optimize(objective, n_trials=50)
    return study.best_params

params = optimize()

In [ ]:
params['random_state'] = 666
model = LGBMClassifier(**params)
model.fit(X, y, categorical_feature=categorical)
preds = model.predict(X_val)
print('LGBM accuracy: ', accuracy_score(y_val, preds)*100, '%')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plot_confusion_matrix(model, X_val, y_val, ax=ax)

In [ ]:
y_preds = pd.Series(model.predict(test.iloc[:,:-1]),name='Stay')

In [ ]:
sub = pd.concat([sample_df.iloc[:,:-1],y_preds],axis=1)

In [ ]:
sub['Stay'] = sub['Stay'].map({0:'0-10',1:'11-20',2:'21-30',3:'31-40',4:'41-50',5:'51-60',6:'61-70',7:'71-80',8:'81-90',
                                9:'91-100',10:'More than 100 Days'})

In [ ]:
sub.head()

In [ ]:
len(sub)

In [ ]:
sub.to_csv("optuna_lgb.csv",index=False)

In [ ]:
import xgboost as xb
import lightgbm as lbm
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
def fill_null(df):
  df['Bed Grade'].fillna(df['Bed Grade'].value_counts().index[0],inplace=True)
  df['City_Code_Patient'].fillna(df['City_Code_Patient'].value_counts().index[0],inplace=True)

In [ ]:
# filling null values of train data

fill_null(train_df)

# filling null values of test data 

fill_null(test_df)

In [ ]:
def convert_to_numerical(df):
  dept_list = df['Department'].unique()
  dept_list.sort()
  dept_dict = dict(zip(dept_list, range(len(dept_list))))
  df['Department'].replace(dept_dict, inplace=True)


  hrc_list = df['Hospital_region_code'].unique()
  hrc_list.sort()
  hrc_dict = dict(zip(hrc_list, range(len(hrc_list))))
  df['Hospital_region_code'].replace(hrc_dict, inplace=True)

  ward_list = df['Ward_Type'].unique()
  ward_list.sort()
  ward_dict = dict(zip(ward_list, range(len(ward_list))))
  df['Ward_Type'].replace(ward_dict, inplace=True)

  wfc_list = df['Ward_Facility_Code'].unique()
  wfc_list.sort()
  wfc_dict = dict(zip(wfc_list, range(len(wfc_list))))
  df['Ward_Facility_Code'].replace(wfc_dict, inplace=True)

  toa_list = df['Type of Admission'].unique()
  toa_list.sort()
  toa_dict = dict(zip(toa_list, range(len(toa_list))))
  df['Type of Admission'].replace(toa_dict, inplace=True)

  soi_list = df['Severity of Illness'].unique()
  soi_list.sort()
  soi_dict = dict(zip(soi_list, range(len(soi_list))))
  df['Severity of Illness'].replace(soi_dict, inplace=True)

  age_list = df['Age'].unique() 
  age_list.sort()
  age_dict = dict(zip(age_list, range(len(age_list))))
  df['Age'].replace(age_dict, inplace=True)

  htc_list = df['Hospital_type_code'].unique()
  htc_list.sort()
  htc_dict = dict(zip(htc_list, range(len(htc_list))))
  df['Hospital_type_code'].replace(htc_dict, inplace=True)

In [ ]:
# converting categorical features of train data into numerical

convert_to_numerical(train_df)

# converting categorical features of test data into numerical

convert_to_numerical(test_df)

In [ ]:
# converting categorical target feature into numerical

stay_list = train_df['Stay'].unique()
stay_list.sort()
stay_dict = dict(zip(stay_list, range(len(stay_list))))
train_df['Stay'].replace(stay_dict, inplace=True)

In [ ]:
#Choosing dependent and independent features 
# selecting independent features and target feature

X = train_df.drop(columns=['Stay', 'case_id','patientid'])
Y = train_df['Stay']


# selecting features for test data 

test_X = test_df.drop(columns=['case_id','patientid'])

In [ ]:
#Preparing training data for CatBoost ¶
# splitting into training and validation data

X_train , X_test , Y_train , Y_test = train_test_split(X,Y,test_size = 0.2 , random_state = 0)
# preparing train and validation data 

train_dataset = Pool(data=X_train, label=Y_train)

eval_dataset = Pool(data=X_test, label=Y_test)

In [ ]:
#Modelling 
# initialising catboost classifier

model = CatBoostClassifier(iterations=750,
                           learning_rate=0.08,
                           depth=7,
                           loss_function='MultiClass',
                           eval_metric='Accuracy')

In [ ]:
# fitting the model

model.fit(train_dataset)

# validation

eval_pred = model.predict(eval_dataset)

In [ ]:
#Accuracy and confusion matrix 
# accuracy

model.get_best_score()

In [ ]:
# confusion matrix

cm = confusion_matrix(Y_test, eval_pred)
cm

In [ ]:
#Preparing testing data for CatBoost 
# preparing test data

test_dataset = Pool(test_X)

In [ ]:
#Testing 
# predictions on test data 

y_pred = model.predict(test_dataset)

In [ ]:
#Output dataframe 
# preparing output dataframe

output = pd.DataFrame(test_df['case_id'].values,columns=['case_id'])
output['Stay'] = y_pred
swap_dict_stay = dict([(value, key) for key, value in stay_dict.items()])
output['Stay'].replace(swap_dict_stay, inplace=True)

In [ ]:
# saving output to csv file

output.to_csv('Healthcare_Submission.csv',index=False)

In [ ]:
output['Stay'].value_counts()